In [1]:
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00


### Load model

In [3]:
import os
from google.colab import userdata
from groq import Groq

client = Groq(
    api_key = userdata.get('GROQ_API_KEY')
)
model = "llama3-70b-8192"

### Create json with results of 8 UCL

In [7]:
import json

def get_ucl_quarter_final_score(team_name):
  """Get the score for a given Football game"""
  if "arsenal" in team_name.lower():
    return json.dumps(
        {
            "game_id": "1", "status": "quarter_final",
            "home_team": "Arsenal", "home_team_score": 2,
            "away_team": "Bayern Munich", "away_team_score": 3
        }
    )
  elif "real" in team_name.lower():
    return json.dumps(
        {
            "game_id": "2", "status": "quarter_final",
            "home_team": "Real Madrid", "home_team_score": 4,
            "away_team": "Manchester City", "away_team_score": 4
        }
    )
  elif "paris" in team_name.lower():
    return json.dumps(
        {
            "game_id": "3", "status": "quarter_final",
            "home_team": "Barcelona", "home_team_score": 4,
            "away_team": "Paris Saint-German", "away_team_score": 6
        }
    )
  elif "Atletico" in team_name.lower():
    return json.dumps(
        {
            "game_id": "4", "status": "quarter_final",
            "home_team": "Borussia Dortmund", "home_team_score": 4,
            "away_team": "Atletico de Madrid", "away_team_score": 5
        }
    )
  else:
    return json.dumps(
        {
            "team_name": team_name,
            "score": "Does not exist"
        }
    )

In [8]:
def run_conversation(user_prompt):
  # Step 1: send the conversation and available functions to the model
  messages = [
      {
          "role": "system",
          "content": """You are a function calling LLM that uses the data extracted from the get_ucl_quarter_final_score function to answer questions around football game scores.
            Include the team and their opponent in your response."""
      },
      {
          "role": "user",
          "content": user_prompt
      }
  ]

  tools = [
      {
          "type": "function",
          "function": {
              "name": "get_ucl_quarter_final_score",
              "description": "Get the score for a given Football game",
              "parameters": {
                  "team_name": {
                      "type": "string",
                      "description": "The name of the Football team (e.g. 'Bayern Munich')",
                  }
              },
              "required": ["team_name"],
          },
      }
  ]
  response = client.chat.completions.create(
        model = model,
        messages = messages,
        tools = tools,
        tool_choice = "auto",
        max_tokens = 4096
  )

  response_message = response.choices[0].message
  print(f"Initial Response: {response_message} \n")
  tool_calls = response_message.tool_calls

  print(f"{tool_calls} \n")
  # Step 2: check if the model wanted to call a function
  print("Final Response:")
  if tool_calls:
      # Step 3: call the function
      available_functions = {
          "get_ucl_quarter_final_score": get_ucl_quarter_final_score,
      }

      messages.append(response_message)
      # Step 4: send the info for each function call and function response to the model
      for tool_call in tool_calls:
          function_name = tool_call.function.name
          function_to_call = available_functions[function_name]
          function_args = json.loads(tool_call.function.arguments)
          function_response = function_to_call(
              team_name=function_args.get("team_name")
          )
          messages.append(
              {
                  "tool_call_id": tool_call.id,
                  "role": "tool",
                  "name": function_name,
                  "content": function_response,
              }
          )  # extend conversation with function response
      second_response = client.chat.completions.create(
          model = model,
          messages = messages
      )  # get a new response from the model where it can see the function response
      return second_response.choices[0].message.content
  else:
      return response.choices[0].message.content

In [9]:
user_prompt = "What was the score of the Arsenal game?"
print(run_conversation(user_prompt))

Initial Response: ChoiceMessage(content=None, role='assistant', tool_calls=[ChoiceMessageToolCall(id='call_vkkc', function=ChoiceMessageToolCallFunction(arguments='{"team_name":"Arsenal"}', name='get_ucl_quarter_final_score'), type='function')]) 

[ChoiceMessageToolCall(id='call_vkkc', function=ChoiceMessageToolCallFunction(arguments='{"team_name":"Arsenal"}', name='get_ucl_quarter_final_score'), type='function')] 

Final Response:
The score of the Arsenal game is: Arsenal 2, Bayern Munich 3.


In [10]:
user_prompt = "What was the score of the Real Madrid game?"
print(run_conversation(user_prompt))

Initial Response: ChoiceMessage(content=None, role='assistant', tool_calls=[ChoiceMessageToolCall(id='call_e9p4', function=ChoiceMessageToolCallFunction(arguments='{"team_name":"Real Madrid"}', name='get_ucl_quarter_final_score'), type='function')]) 

[ChoiceMessageToolCall(id='call_e9p4', function=ChoiceMessageToolCallFunction(arguments='{"team_name":"Real Madrid"}', name='get_ucl_quarter_final_score'), type='function')] 

Final Response:
According to the UCL quarter-final score, Real Madrid played against Manchester City and the score was Real Madrid 4, Manchester City 4.


In [11]:
user_prompt = "What was the score of the Chelsea game?"
print(run_conversation(user_prompt))

Initial Response: ChoiceMessage(content=None, role='assistant', tool_calls=[ChoiceMessageToolCall(id='call_15xp', function=ChoiceMessageToolCallFunction(arguments='{"team_name":"Chelsea"}', name='get_ucl_quarter_final_score'), type='function')]) 

[ChoiceMessageToolCall(id='call_15xp', function=ChoiceMessageToolCallFunction(arguments='{"team_name":"Chelsea"}', name='get_ucl_quarter_final_score'), type='function')] 

Final Response:
It seems that Chelsea did not make it to the quarterfinals of the UEFA Champions League. As a result, I don't have any score information for Chelsea in the quarterfinals.
